In [1]:
import torch 

x = torch.arange(5.0)
x

tensor([0., 1., 2., 3., 4.])

In [16]:
x.requires_grad_(True) # 等价于 x = torch.arange(5.0, requires_grad=True)
# x.grad # 默认值是None
y = 2 * torch.dot(x, x)
y

tensor(60., grad_fn=<MulBackward0>)

In [17]:
# 接下来，通过调用反向传播函数来自动计算y关于x每个分量的梯度
y.backward() # y对x求导
x.grad

tensor([ 0.,  4.,  8., 12., 16.])

In [18]:
x.grad == 4 * x

tensor([True, True, True, True, True])

In [19]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1., 1.])

In [20]:
# 非标量变量的反向传播
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6., 8.])

In [21]:
# 分离计算
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True, True])

In [22]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x


tensor([True, True, True, True, True])

In [23]:
# Python控制流的梯度计算，使用自动微分的一个好处是： 即使构建函数的计算图需要通过Python控制流
# （例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度。 在下面的代码中，while循环的迭代次数和if语句的结果都取决于输入a的值。
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [ ]:
# 我们现在可以分析上面定义的f函数。 请注意，它在其输入a中是分段线性的。 
# 换言之，对于任何a，存在某个常量标量k，使得f(a)=k*a，其中k的值取决于输入a，因此可以用d/a验证梯度是否正确。
a.grad == d / a

tensor(True)